In [3]:
# importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob

In [4]:
import os 
os.getcwd()

'D:\\UIBee\\Mask_RCNN\\samples\\uibee'

In [8]:
# loading the directories 
training_dir = 'elements/train'
validation_dir = 'elements/val'

In [9]:
# useful for getting number of files
image_files = glob(training_dir + '/*.*')
valid_image_files = glob(validation_dir + '/*.*')

In [10]:
# getting the number of classes i.e. type of fruits
folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

Total Classes = 14


In [12]:
import tensorflow as tf
device_name=tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
tf.test.gpu_device_name()

Found GPU at: /device:GPU:0


'/device:GPU:0'

In [15]:
# this will copy th# importing the libraries
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications import VGG16
#from keras.preprocessing import image

IMAGE_SIZE = [224, 224]  # we will keep the image size as (64,64). You can increase the size for better results. 

# loading the weights of VGG16 without the top layer. These weights are trained on Imagenet dataset.
vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  # input_shape = (64,64,3) as required by VGG

# this will exclude the initial layers from training phase as there are already been trained.
for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
#x = Dense(128, activation = 'relu')(x)   # we can add a new fully connected layer but it will increase the execution time.
x = Dense(num_classes, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.

model = Model(inputs = vgg.input, outputs = x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [16]:
# Image Augmentation

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input

training_datagen = ImageDataGenerator(
                                    rescale=1./255,   # all pixel values will be between 0 an 1
                                    shear_range=0.2, 
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    preprocessing_function=preprocess_input)

validation_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function=preprocess_input)

training_generator = training_datagen.flow_from_directory(training_dir, target_size = IMAGE_SIZE, batch_size = 200, class_mode = 'categorical')
validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size = IMAGE_SIZE, batch_size = 200, class_mode = 'categorical')

Found 5570 images belonging to 14 classes.
Found 878 images belonging to 14 classes.


In [17]:
# The labels are stored in class_indices in dictionary form. 
# checking the labels
training_generator.class_indices

{'Box': 0,
 'Button': 1,
 'Button - Plus': 2,
 'Checkbox': 3,
 'Dropdown': 4,
 'Grid': 5,
 'Image': 6,
 'Input': 7,
 'Paragraph': 8,
 'Radio Button': 9,
 'Slider': 10,
 'Table': 11,
 'Title': 12,
 'Toggle': 13}

In [18]:
training_images = 5640
validation_images = 878

history = model.fit_generator(training_generator,
                   steps_per_epoch = 5640,  # this should be equal to total number of images in training set. But to speed up the execution, I am only using 10000 images. Change this for better results. 
                   epochs = 1,  # change this for better results
                   validation_data = validation_generator,
                   validation_steps = 878)  # this should be equal to total number of images in validation set.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/1
5640/5640 [==============================] - 9440s 2s/step - loss: 0.0165 - acc: 0.9961 - val_loss: 3.3582e-04 - val_acc: 1.0000


In [27]:
print ('Training Accuracy = ' + str(history.history['acc']))
print ('Training Loss = ' + str(history.history['loss']))
print ('Validation Accuracy = ' + str(history.history['val_acc']))
print ('Validation Loss = ' + str(history.history['val_loss']))

Training Accuracy = [0.9961379409851876]
Training Loss = [0.016500808591820033]
Validation Accuracy = [1.0]
Validation Loss = [0.0003358220102684723]
